In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
input_dim_h = 512
input_dim_w = 512

transform = transforms.Compose([
                    transforms.Resize((input_dim_w, input_dim_h)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root = "/content/drive/My Drive/Colab Notebooks/CS221/dataset"
full_set = datasets.ImageFolder(root=root, transform=transform)
train_size = int(0.8 * len(full_set))
test_size = len(full_set) - train_size
train_set, test_set = torch.utils.data.random_split(full_set, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=0)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),  # (1024, 1024, 3) -> (32, 32, 32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  # (512, 512, 32)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),  # (16, 16, 64)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  #  (256, 256, 64)
        # self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(128 * 128 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        # out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

model = ConvNet()

In [0]:
import torch.optim as optim

# Define the loss function and optimizer
loss_fn = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [0]:
for epoch in range(1):
    # Train the model
    running_loss = 0.0
    model.train()
    for i, [inputs, labels] in enumerate(train_loader):   
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
        
    # Test current version of model to obtain accuracy    
    correct = total = 0 
    with torch.no_grad():
        for [inputs, labels] in test_loader:   
            probs = model(inputs)
            t = torch.Tensor([0.5])  # 0.5 acts as threshold
            outputs = (probs > t).float() * 1
            total += len(outputs)
            correct += (outputs == labels.float()).float().sum()
            
    accuracy = correct / total
    print ('Loss: {}     Accuracy: {}'.format(running_loss,  accuracy))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 89 %


In [0]:
print (len(train_loader))
print (len(test_loader))

1280
320
